# [INFO-H515 - Big Data Scalable Analytics](http://uv.ulb.ac.be/course/view.php?id=74317?username=guest)


## TP 4 - Recommender system with low rank matrix approximation and Alternating Least Squares

#### *Yann-Aël Le Borgne, Jacopo De Stefani and Gianluca Bontempi*

####  09/05/2018


This class aims at implementing the Alternating Least Square (ALS) algorithm, a popular machine learning technique for recommendation systems. 

We will use the MovieLens dataset (https://grouplens.org/datasets/movielens/) for making recommendation on movies. The data is available from this notebook folder.

### Class objectives:

* Implementation of ALS in numpy
* Implementation of ALS using Map/Reduce
* Implementation of ALS using Spark ML

# Recommender systems

* Recommender systems have become increasingly popular in recent years, and aim at inferring customer's preference based on their past ratings, as well as the past ratings of other customers.  
* They are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general. 
* The ratings provided by a customer usually involve a very small number of products.
* The problem can be formalized as a matrix completion problem, where the goal is to predict the (many) missing values of a rating matrix. 
* An efficient approach to solve this problem is to rely on *low rank matrix approximation*, and the *Alternating Least Square (ALS)* algorithm.



### Notations
* Let $n_u$ and $n_p$ be the number of users and products.
* Let $R \in \mathbb{R}^{n_u \times n_p}$ be the matrix of ratings, where entry $r_{ij}$, $1 \le i \le n_u$ and $1 \le j \le n_p$, is the rating of user $i$ for product $j$. Entries $r_{ij}$ contain many missing values.
* Let $w_{i,j}$ be an indicator of the existence of a rating for product $j$ by user $i$, i.e., $w_{i,j}=1$ if the rating $(i,j)$ exists, and $w_{i,j}=0$ otherwise.
* Let $k$ be the rank of the matrix factorisation.
* Let $U \in \mathbb{R}^{n_u \times n_k}$ be the user matrix, and $P \in \mathbb{R}^{n_p \times n_k}$ be the products (item) matrix 
* Let $U_i$ be the $i-$th row of $U$, and $P_j$ be the $j-$th row of $P$.

![alt text](mat_prod.jpg "Ratings matrix factorisation")

* Let $\hat{R}$ be the predicted rating matrix, where all missing values are predicted on the basis of known user ratings. 
* The optimisation function is expressed as 

$$
J(U,P)=||(R-UP^T)||_2
$$

and predictions $\hat{R}$ given by 

$$
\hat{R}=UP^T
$$

### Alternating Least Squares

* Note that since both $U$ and $P$ are unknown, the optimisation function $J(U,V)$ is non convex.
* However, if we fix $P$ and optimise for $U$ alone, the problem is simply reduced to the problem of linear regression, and can be solved using Ordinary Least Square (OLS):

$$
U^T=(P^TP)^{-1}P^TR^T
$$

Then, fixing $U$ and optimising for P gives

$$
P^T=(U^TU)^{-1}U^TR
$$


* ALS does just that, iteratively optimising $U$ by fixing $P$, and optimising $P$ by fixing $U$. It is guaranteed to converge only to a local minima, which ultimately depends on initial values for $U$ or $P$. 

* **Missing values**: Since R contains missing values, regression must be computed per user (or product), using only those entries for which the ratings are known ($w_{i,j}=1$). This is done by going though all users (or products):
    * For each i, compute $$U^T_i=(\sum_{j, w_{i,j}=1} P_j^TP_j)^{-1} \sum_{j, w_{i,j}=1} P_j^Tr_{ij}$$
    * For each j, compute $$P^T_j=(\sum_{i, w_{i,j}=1} U_i^TU_i)^{-1} \sum_{i, w_{i,j}=1} U_i^Tr_{ij}$$


**ALS algorithm:**

1. Initialize the matrix P by assigning to the first column the average rating for each product, and using small random numbers for the remaining columns.
2. Fix P and solve for U_i that minimizes the objective function (the root mean square error (RMSE)).
3. Fix U and solve for P_j that minimizes the objective function similarly.
4. Repeat steps 2 and 3 until convergence


# General imports

In [1]:
import time
import os 
import numpy as np
import pandas as pd

%matplotlib notebook  
import matplotlib as mpl
import matplotlib.pyplot as plt

# Load data

Let us first load the data, which is in CSV format in `ml-latest-small/ratings.csv`. 


In [2]:
rating_df = pd.read_csv( "ml-latest-small/ratings.csv")

Each row of the dataset provides the rating given by a user to a movies, as well as a timestamp. There are 100004 ratings in total

In [3]:
rating_df[0:5]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
rating_df.shape

(100004, 4)

### Data preprocessing

Data preprocessing consists in :
* Dropping the timestamp column, which will not be used
* Creating a training and validation set (random 80/20% split of the original data)
* Reindexing the userId and movieId so their range is in the interval [0,n\_u-1] and [0,n_p-1]
* Finally, removing from the validation set user IDs and movie IDs that do not appear in the training set (we will not deal with the problem of rating new users/movies).

In [5]:
#Let us drop the timestamp since we will not use it.
rating_df=rating_df.drop( "timestamp", axis = 1 )

#Create training and validation set
np.random.seed(0)
i_training = np.random.rand(len(rating_df)) < 0.8
training_df= rating_df[i_training].reset_index(drop=True)
validation_df= rating_df[~i_training].reset_index(drop=True)

#Keep only those movies and user IDs which are in the training data (no predictions for 'new' users and movies)
validation_df=validation_df[validation_df.movieId.isin(training_df.movieId)]
validation_df=validation_df[validation_df.userId.isin(training_df.userId)].reset_index(drop=True)

#Reindex users and movies so IDs are between 0 and numbers of users/movies
userId=training_df.userId.unique()
user_mapping=dict(zip(userId,range(len(userId))))
training_df.userId=training_df.userId.map(user_mapping)
validation_df.userId=validation_df.userId.map(user_mapping)

movieId=training_df.movieId.unique()
movie_mapping=dict(zip(movieId,range(len(movieId))))
training_df.movieId=training_df.movieId.map(movie_mapping)
validation_df.movieId=validation_df.movieId.map(movie_mapping)


After preprocessing, we have
* 79962 ratings in the training set
* 19281 ratings in the validation set
* 671 unique user IDs, and 8369 unique movie IDs 

In [6]:
training_df.shape

(79962, 3)

In [7]:
validation_df.shape

(19281, 3)

In [8]:
#Number of unique users
n_u=len( training_df.userId.unique())
n_u

671

In [9]:
#Number of unique movies
n_p=len( training_df.movieId.unique())
n_p

8369

In [10]:
training_df[0:3]

,userId,movieId,rating
0,0,0,2.5
1,0,1,3.0
2,0,2,3.0


In [11]:
validation_df[0:3]

,userId,movieId,rating
0,0,676,2.0
1,0,680,3.5
2,0,2256,4.0


# 1) Baseline model - Centralised approach

## 1.1) Constant model

Let us first make a centralised constant model, using Pandas and numpy, where the predicted score for any movie is set to 2.5

To quantify the error of the prediction, let us use the Root Mean Squard Error (RMSE), which is the root of the average quared difference between the prediction and the true value. 

In [12]:
#Compute the RMSE (root mean square error) between true ratings and predictions
def computeRMSE(ratings,predictions):
    
    RMSE=np.sqrt(np.sum((predictions-ratings)**2)/len(predictions))
    
    return RMSE


**Exercise**

Compute the prediction error on the training set using the constant model

In [13]:
ratings=np.array(training_df.rating)
N=ratings.shape[0]
predictions=np.full((N),2.5)

In [14]:
computeRMSE(ratings,predictions)

1.4863354657446775

**Exercise**

Compute the prediction error on the validation set using the constant model

In [15]:
ratings=np.array(validation_df.rating)
N=ratings.shape[0]
predictions=np.full((N),2.5)

In [16]:
computeRMSE(ratings,predictions)

1.489508283150651

## 1.2) Average model

Let us now make an 'average' model, where the rating prediction for a movie is the average rating for that movie in the training set.

**Exercise**

1. Compute the average rating for each movie in the training set (use the groupby and mean function from Pandas)
2. Compute the prediction error on the training set using the average model
3. Compute the prediction error on the validation set using the average model

In [17]:
average_rating_movie=training_df.groupby('movieId').rating.mean()

In [18]:
training_predictions=average_rating_movie[training_df.movieId]

In [19]:
computeRMSE(np.array(training_df.rating),np.array(training_predictions))

0.88893244960704643

In [20]:
validation_predictions=average_rating_movie[validation_df.movieId]

In [21]:
computeRMSE(np.array(validation_df.rating),np.array(validation_predictions))

0.99531137099735412

Note that the prediction error is higher on the validation set (overfitting)

# 2) ALS - Centralised approach

Let us follow the alogrithm provided in the introduction section to compute the ALS predictions:

### Initialisation

* Initialise k (the ALS rank), and lambda_reg (the regularisation parameter)
* Initialize the matrix P by assigning to the first column the average rating for each movie, and using small random numbers for the remaining columns.

In [22]:
k=1

np.random.seed(0)
P=np.random.rand(n_p,k)

average_rating_movie=np.array(training_df.groupby('movieId').rating.mean())
P[:,0]=average_rating_movie

In [23]:
n_p

8369

### OLS function

The function aims at computing the OLS for a subset of entries of the rating matrix. It takes as inputs
* The set of indices to keep from the rating matrix, as well as the corresponding ratings. This is provided as an array of two columns.
* The U or P matrix, depending on which matrix is updated. This is provided as an array, called X.

In [24]:
def OLS(list_id_rating, X,lambda_reg=0.1):
    list_id_rating=np.reshape(np.array(list_id_rating),(-1,2))
    X=np.array(X)
    
    #Get the subset of rows from X for which to compute OLS
    #Need to first convert indices to integers 
    list_id=[int(elt) for elt in list_id_rating[:,0]]
    X_j=X[list_id,:] 
    
    #Compute OLS
    k=X_j.shape[1]
    XtX=np.dot(np.transpose(X_j),X_j)+lambda_reg*np.identity(k)
    XtY=np.dot(np.transpose(X_j),list_id_rating[:,1])
    coefficient_OLS=np.transpose(np.dot(np.linalg.inv(XtX),XtY))
    
    return coefficient_OLS


### ALS iterations

In [25]:
#Number of iterations
T=1

for t in range(T):
    
    print('Iteration: ',t)
    
    U=np.vstack(training_df.groupby('userId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['movieId','rating']],P)))
    
    P=np.vstack(training_df.groupby('movieId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['userId','rating']],U)))
    
    


Iteration:  0


In [26]:
#Check results of groupby/apply transform
#np.vstack(training_df.groupby('userId').\
#                            apply(lambda list_id_rating: OLS(list_id_rating[['movieId','rating']],P)))[0:4,:]

### Get rating predictions and assess model performances

In [27]:
#This takes U and P matrices, and compute the predictions for pairs userId/movieId in rating_df
def getPredictions(U,P,rating_df):
    N=rating_df.shape[0]
    predictions=np.zeros(N)
    
    for i in range(N):
        predictions[i]=np.sum(np.dot(U[rating_df.userId[i],:],P[rating_df.movieId[i],:]))
        
    return predictions


In [28]:
pred_training=getPredictions(U,P,training_df)
computeRMSE(np.array(training_df.rating),np.array(pred_training))

0.80989152206399284

In [29]:
pred_validation=getPredictions(U,P,validation_df)
computeRMSE(np.array(validation_df.rating),np.array(pred_validation))

0.90748126399211981

**Exercise**

Modify the main loop of ALS to print the RMSE on the trainnig and validation set at each iteration. Modify k, lambda_reg, and T, and see how that influences the predictions. What seems to be the best parameters?

In [30]:
k=1

np.random.seed(0)
P=np.random.rand(n_p,k)
average_rating_movie=np.array(training_df.groupby('movieId').rating.mean())
P[:,0]=average_rating_movie

#Number of iterations
T=1

for t in range(T):
    
    print('Iteration: ',t)
    
    U=np.vstack(training_df.groupby('userId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['movieId','rating']],P)))
    
    P=np.vstack(training_df.groupby('movieId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['userId','rating']],U)))
    
    training_predictions=getPredictions(U,P,training_df)
    training_error=computeRMSE(np.array(training_df.rating),np.array(training_predictions))
    
    print("Error training set: "+str(training_error))
    
    validation_predictions=getPredictions(U,P,validation_df)
    validation_error=computeRMSE(np.array(validation_df.rating),np.array(validation_predictions))
    
    print("Error validation set: "+str(validation_error))
    


Iteration:  0
Error training set: 0.809891522064
Error validation set: 0.907481263992


# 3) ALS - Map/Reduce approach

### Start Spark context

In [31]:
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"

from pyspark.sql import SparkSession

#Start Spark session with local master and 2 cores
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("ALS") \
    .getOrCreate()

sc=spark.sparkContext

### Transform training and validation in RDD

In [32]:
#Number of partitions
p=4

training_df_rdd=sc.parallelize(np.array(training_df),p)
validation_df_rdd=sc.parallelize(np.array(validation_df),p)

Same initialisation as centralised ALS

In [33]:
k=1

np.random.seed(0)
P=np.random.rand(n_p,k)

average_rating_movie=np.array(training_df.groupby('movieId').rating.mean())
P[:,0]=average_rating_movie

**Exercise**

Adapt the main loop of the centralised ALS in a Spark Map/Reduce way.

The centraliseed implementation is:

```
for t in range(T):
    
    print('Iteration: ',t)
    
    U=np.vstack(training_df.groupby('userId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['movieId','rating']],P)))
    
    P=np.vstack(training_df.groupby('movieId').
                            apply(lambda list_id_rating: OLS(list_id_rating[['userId','rating']],U)))

```

Steps:

* `training_df.groupby('userId')`: This part should be transformed so that `training_df_rdd` is first map in (key,value) pairs where th key is the userId, and the value the tuple (movieId,rating)
* From the RDD obtained in the previous step, group by key
* Apply the `OLS` function on the group by result using the map operator
* Sort the result by key (using `sortByKey`)
* Get the values (the result of the OLS function), using the `values`function
* Collect()

In [34]:
T=1

for t in range(T):
    
    print('Iteration: ',t)
    
    #Grouped by user ID (first column), value is (movieId,rating) (second and third columns)
    R1=training_df_rdd.map(lambda x: (x[0],[x[1],x[2]])).groupByKey()
    
    U=np.array(R1.mapValues(lambda list_id_rating:OLS(list(list_id_rating),P)).
                    sortByKey().values().collect())
    
    #Grouped by movie ID (second column), value is (movieId,rating) (first and third columns)
    R2=training_df_rdd.map(lambda x: (x[1],[x[0],x[2]])).groupByKey()
    P=np.array(R2.mapValues(lambda list_id_rating:OLS(list(list_id_rating),U)).
                    sortByKey().values().collect())
    
    training_predictions=getPredictions(U,P,training_df)
    training_error=computeRMSE(np.array(training_df.rating),np.array(training_predictions))
    
    print("Error training set: "+str(training_error))
    
    validation_predictions=getPredictions(U,P,validation_df)
    validation_error=computeRMSE(np.array(validation_df.rating),np.array(validation_predictions))
    
    print("Error validation set: "+str(validation_error))
    

Iteration:  0
Error training set: 0.809891522064
Error validation set: 0.907481263992


Let us get the RMSE on the training and validation sets

In [35]:
pred_training=getPredictions(U,P,training_df)
computeRMSE(np.array(training_df.rating),np.array(pred_training))

0.80989152206399284

In [36]:
pred_validation=getPredictions(U,P,validation_df)
computeRMSE(np.array(validation_df.rating),np.array(pred_validation))

0.90748126399211981

In [37]:
pred_training[0:10]

array([ 2.26142018,  2.58954558,  2.43693271,  2.26011812,  3.00573893,
        2.75452198,  2.74514419,  2.55343655,  2.1336082 ,  2.10186937])

In [38]:
training_df.rating[0:10]

0    2.5
1    3.0
2    3.0
3    2.0
4    4.0
5    2.0
6    2.0
7    2.0
8    2.5
9    1.0
Name: rating, dtype: float64

### Optimisation

Note that R1 and R2 do not need to be computed at each iteration. They can be computed before, and cached for re-use

In [39]:
k=1

np.random.seed(0)
P=np.random.rand(n_p,k)

average_rating_movie=np.array(training_df.groupby('movieId').rating.mean())
P[:,0]=average_rating_movie



R1=training_df_rdd.map(lambda x: (x[0],[x[1],x[2]])).groupByKey().cache()
R2=training_df_rdd.map(lambda x: (x[1],[x[0],x[2]])).groupByKey().cache()

T=1

for t in range(T):
    
    print('Iteration: ',t)
    
    #Grouped by user ID (first column), value is (movieId,rating) (second and third columns)
    R1=training_df_rdd.map(lambda x: (x[0],[x[1],x[2]])).groupByKey()
    
    U=np.array(R1.mapValues(lambda list_id_rating:OLS(list(list_id_rating),P)).
                    sortByKey().values().collect())
    
    #Grouped by movie ID (second column), value is (movieId,rating) (first and third columns)
    R2=training_df_rdd.map(lambda x: (x[1],[x[0],x[2]])).groupByKey()
    P=np.array(R2.mapValues(lambda list_id_rating:OLS(list(list_id_rating),U)).
                    sortByKey().values().collect())
    
    training_predictions=getPredictions(U,P,training_df)
    training_error=computeRMSE(np.array(training_df.rating),np.array(training_predictions))
    
    print("Error training set: "+str(training_error))
    
    validation_predictions=getPredictions(U,P,validation_df)
    validation_error=computeRMSE(np.array(validation_df.rating),np.array(validation_predictions))
    
    print("Error validation set: "+str(validation_error))


Iteration:  0
Error training set: 0.809891522064
Error validation set: 0.907481263992


# 4) ALS with Spark ML library 

See documentation at https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [40]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


Let us use the same parameters as our implementation

In [41]:
MAX_ITERATIONS = 10
REG_PARAM = 0.1
SEED_VALUE = 0

Create ALS object

In [42]:
als = ALS()
als.setMaxIter(MAX_ITERATIONS)          \
 .setSeed(SEED_VALUE)                 \
 .setRegParam(REG_PARAM)              \
 .setUserCol('userId')                \
 .setItemCol('movieId')               \
 .setRatingCol('rating')

ALS_479ba984b099902f858f

Create an RMSE evaluator using the label and predicted columns

In [43]:
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")

Convert training and validation sets in Spark dataframes

In [44]:
training_df_rdd=spark.createDataFrame(training_df, ['userId', 'movieId', 'rating']) 
validation_df_rdd=spark.createDataFrame(validation_df, ['userId', 'movieId', 'rating']) 

Train an ALS model with ranks of increasing sizes

In [45]:
ranks = [1,2,5,10,20]
errors_training = []
errors_validation = []
models = []

min_error = float('inf')
best_rank = -1

for i in range(len(ranks)):

  # Build the model    
  als.setRank(ranks[i])
  model = als.fit(training_df_rdd)

  # Make predictions on training dataset  
  training_predictions_df=model.transform(training_df_rdd)
  # Run the previously created RMSE evaluator, reg_eval, on the training_predictions_df DataFrame
  errors_training.append(reg_eval.evaluate(training_predictions_df))

  # Make predictions on validation dataset  
  validation_predictions_df=model.transform(validation_df_rdd)
  # Run the previously created RMSE evaluator, reg_eval, on the validation_predictions_df DataFrame
  errors_validation.append(reg_eval.evaluate(validation_predictions_df))

  print( 'For rank %s the training RMSE is %s' % (ranks[i], errors_training[i]) )
  print( 'For rank %s the validation RMSE is %s' % (ranks[i], errors_validation[i]) )

  if errors_validation[i] < min_error:
      min_error = errors_validation[i]
      best_rank = ranks[i]
  
print( 'The best model was trained with rank %s' % best_rank )

For rank 1 the training RMSE is 0.8116843908249538
For rank 1 the validation RMSE is 0.9118534800447594
For rank 2 the training RMSE is 0.7506271366944117
For rank 2 the validation RMSE is 0.9071371402911167
For rank 5 the training RMSE is 0.6594351988007074
For rank 5 the validation RMSE is 0.9170910150192021
For rank 10 the training RMSE is 0.5800020605088405
For rank 10 the validation RMSE is 0.9193936976666198
For rank 20 the training RMSE is 0.5111833971151686
For rank 20 the validation RMSE is 0.9143466118176342
The best model was trained with rank 2


In [46]:
#Computing error using computeRMSE
validation_predictions=validation_predictions_df.toPandas()

In [47]:
validation_predictions[0:4]

,userId,movieId,rating,prediction
0,133,148,4.0,3.691487
1,362,148,4.0,3.370282
2,285,148,4.0,3.029837
3,325,148,4.0,3.093536


In [48]:
#This should match the RMSE obtained for the last ALS model above
computeRMSE(np.array(validation_predictions.rating),np.array(validation_predictions.prediction))

0.91434661181763421

### References

* Koren, Yehuda, Robert Bell, and Chris Volinsky. "Matrix factorization techniques for recommender systems." Computer 42.8 (2009). https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf
* http://www.awesomestats.in/spark-movie-recommendations/
* https://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf
* https://en.wikipedia.org/wiki/Recommender_system
* https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/
* https://stanford.edu/~rezab/classes/cme323/S16/projects_reports/parthasarathy_tea.pdf
* https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/mllib/recommendation/ALS.scala
* http://yifanhu.net/PUB/cf.pdf
* https://github.com/bdanalytics/Berkeley-Spark/blob/master/CS110x/cs110_lab2_als_prediction.ipynb
* https://www.youtube.com/watch?v=RcOUXmCAssg&list=PL0Smm0jPm9WcCsYvbhPCdizqNKps69W4Z&index=95
